In [1]:
%%capture
## compile PyRoss for this notebook
import os
owd = os.getcwd()
os.chdir('../../')
%run setup.py install
os.chdir(owd)

In [2]:
%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt
import pyross
import time 

In [3]:
M  = 2                  # the population has two age groups
N  =  5e4           # and this is the total population

# correct params

beta  = 0.02         # infection rate
gIa   = 1./7            # recovery rate of asymptomatic infectives
gIs   = 1./7            # recovery rate of asymptomatic infectives
alpha = 0.2          # fraction of asymptomatic infectives
fsa   = 0.8          # the self-isolation parameter

# set the age structure
fi = np.array([0.25, 0.75])  # fraction of population in age age group
Ni = N*fi

# set the contact structure
C = np.array([[18., 9.], [3., 12.]])

# set up initial condition
Ia0 = np.array([10, 10])  # each age group has asymptomatic infectives
Is0 = np.array([10, 10])   # and also symptomatic infectives
R0  = np.array([0, 0])  # there are no recovered individuals initially
S0  = Ni - (Ia0 + Is0 + R0)

Tf = 100
Nf = Tf+1
steps = 11

def contactMatrix(t):
    return C

parameters = {'alpha':alpha, 'beta':beta, 'gIa':gIa, 'gIs':gIs,'fsa':fsa}

# use pyross stochastic to generate traj and save 
sto_model = pyross.stochastic.SIR(parameters, M, Ni)
data = sto_model.simulate(S0, Ia0, Is0, contactMatrix, Tf, Nf)
data_array = np.reshape(data['X'], (Tf+1, 3, M))
np.save('sto_traj.npy', data_array)

In [4]:
# load the data and rescale to intensive variables 
x = np.load('sto_traj.npy').astype('float')
x = x/N

# initialise the estimator 
estimator = pyross.inference.SIR(parameters, M, fi, N)

In [5]:
# take a guess 
beta_g = 0.1
gIa_g = 0.1
gIs_g = 0.1
alpha_g = 0.4
guess = [alpha_g, beta_g, gIa_g, gIs_g]

# inference 
start_time = time.time() 
params, nit = estimator.inference(guess, x, Tf, Nf, steps, contactMatrix) # currently only guess four parameters
end_time = time.time()
print(params) # best guess 
print(nit) # number of iterations of the optimization run 
print(end_time - start_time)

[0.19883799 0.02006396 0.14306499 0.12058293]
219
135.61513900756836


In [6]:
# compute -log_p for the original (correct) parameters 
start_time = time.time() 
parameters = {'alpha':alpha, 'beta':beta, 'gIa':gIa, 'gIs':gIs,'fsa':fsa}
logp = estimator.obtain_minus_log_p(parameters, x, Tf, Nf, steps, contactMatrix)
end_time = time.time()
print(logp) 
print(end_time - start_time)

-1885.4537947523097
0.3878180980682373


In [7]:
# compute -log_p for a different set of parameters 
parameters = {'alpha':alpha_g, 'beta':beta_g, 'gIa':gIa_g, 'gIs':gIs_g,'fsa':fsa}
logp = estimator.obtain_minus_log_p(parameters, x, Tf, Nf, steps, contactMatrix)
print(logp) 

63322.78942701757
